字典为动词“to yield”给出了两个释义：产出和让步。对于 Python 生成器 中的 yield 来说，这两个含义都成立。yield item 这行代码会产出一 个值，提供给 next(...) 的调用方；此外，还会作出让步，暂停执行 生成器，让调用方继续工作，直到需要使用另一个值时再调用 next()。调用方会从生成器中拉取值。

In [1]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [8]:
my_coro2 = simple_coro2(14)

In [9]:
my_coro2

<generator object simple_coro2 at 0x000002977D4C33C8>

In [10]:
from inspect import getgeneratorstate

In [11]:
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [12]:
next(my_coro2)

-> Started: a = 14


14

In [13]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [14]:
my_coro2.send(28)

-> Received: b = 28


42

In [15]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [16]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

In [17]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [18]:
coro_avg = averager()

In [19]:
next(coro_avg)

In [20]:
coro_avg.send(10)

10.0

In [21]:
coro_avg.send(30)

20.0

In [22]:
coro_avg.send(5)

15.0

## 预激协程

In [24]:
from functools import wraps

In [25]:
def coroutine(func):
    """装饰器：向前执行到第一个yield表达式，预激func"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [26]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [34]:
coro_avg = averager()

In [29]:
coro_avg.send(10)

10.0

In [31]:
coro_avg.send(30)

20.0

In [32]:
coro_avg.close()

In [33]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

In [36]:
coro_avg.throw(NameError)

NameError: 

In [37]:
def gen():
    yield from 'ab'
    yield from range(1, 3)

In [38]:
list(gen())

['a', 'b', 1, 2]